In [1]:
import pandas as pd
from plotnine.data import mtcars

import numpy as np

from sklearn.preprocessing import StandardScaler

import statistics

import itertools

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
fonte = mtcars[['mpg','disp']].copy()

#fonte = mtcars[['mpg','cyl']].copy()
#fonte = fonte.rename({'cyl': 'disp'}, axis = 1)
#fonte['disp'] = fonte['disp'].astype(float)

quantidade_linhas = 8
fonte = fonte.loc[0:quantidade_linhas-1]
display(fonte)
display(len(fonte))

,mpg,disp
0,21.0,160.0
1,21.0,160.0
2,22.8,108.0
3,21.4,258.0
4,18.7,360.0
5,18.1,225.0
6,14.3,360.0


7

In [3]:
lista_index = []

for i in range(1, quantidade_linhas+1):
    if i < 10:
        lista_index.append("a0"+str(i))
    else:
        lista_index.append("a"+str(i))
    
print(lista_index)

fonte.index = lista_index

['a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07']


In [4]:
for coluna in fonte.columns:
    media = round(fonte[coluna].mean(), 2)
    desvio_padrao = round(fonte[coluna].std(), 2)
    for i in range(0, len(fonte)):
        fonte[coluna][i] = (fonte[coluna][i] - media)/desvio_padrao
        
fonte = fonte

In [5]:
fonte

,mpg,disp
a01,0.489437,-0.734702
a02,0.489437,-0.734702
a03,1.123239,-1.258052
a04,0.630282,0.251610
a05,-0.320423,1.278180
a06,-0.531690,-0.080515
a07,-1.869718,1.278180


In [6]:
df = pd.DataFrame(index=fonte.index, columns = list(fonte.index))
df

,a01,a02,a03,a04,a05,a06,a07
a01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for coluna in df.columns:
    #print(coluna) #cima
    for linha in df.index:
        #print(linha) #baixo
        
        mpg = (fonte['mpg'][coluna] - fonte['mpg'][linha])**2 
        mpg = round(mpg, 2)
        
        disp = (fonte['disp'][coluna] - fonte['disp'][linha])**2
        disp = round(disp, 2)
        
        valor = mpg + disp
        valor = round(valor, 2)
        
        valor = valor/2
        valor = round(valor, 2)
        
        valor = valor**0.5
        valor = round(valor, 2)
        
        
        df[coluna][linha] = valor
        
df_completo = df.copy()
df

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a02,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a03,0.58,0.58,0.0,1.12,2.06,1.44,2.77
a04,0.71,0.71,1.12,0.0,0.99,0.85,1.91
a05,1.54,1.54,2.06,0.99,0.0,0.97,1.1
a06,0.86,0.86,1.44,0.85,0.97,0.0,1.35
a07,2.19,2.19,2.77,1.91,1.1,1.35,0.0


# LOOPING

In [8]:
df = df_completo.copy()
df

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a02,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a03,0.58,0.58,0.0,1.12,2.06,1.44,2.77
a04,0.71,0.71,1.12,0.0,0.99,0.85,1.91
a05,1.54,1.54,2.06,0.99,0.0,0.97,1.1
a06,0.86,0.86,1.44,0.85,0.97,0.0,1.35
a07,2.19,2.19,2.77,1.91,1.1,1.35,0.0


In [9]:
writer = pd.ExcelWriter('distancia_cyl.xlsx', engine='xlsxwriter')

#Tirando as diagonais e repetidos do lado superior direito
dff = df.copy()

lista = []
for coluna in dff.columns:
    for linha in dff.index:
        if coluna == linha:
            dff[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            dff[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna]))) 

dff.to_excel(writer, sheet_name=str(0))

In [10]:
cofenetica = pd.DataFrame(columns = ['Passo','Junção','Nivel'])

count = 0
#count < 5
#len(df.columns) > 2:
while len(df.columns) > 1:
    count += 1 
    
    #Excluindo diagonal
    for coluna in df.columns:
        for linha in df.index:
            if coluna == linha:
                df[coluna][linha] = np.nan

    #Encontrando menor valor na matrix
    menor_valor = 100

    for coluna in df.columns:
        for linha in df.index:
            if df[coluna][linha] == '':
                continue
            elif df[coluna][linha] < menor_valor:
                menor_valor = df[coluna][linha]
                ind_coluna = coluna
                ind_linha = linha

    juntar = ','.join(sorted(list([ind_coluna, ind_linha])))
    separar = juntar.split(',')
    
    juntar2 = ' - '.join(list([ind_coluna, ind_linha]))
    cofenetica.loc[len(cofenetica)] = [count, juntar2, menor_valor]

    #obs: 1 coluna, 0 linha
    df.drop(ind_coluna, axis=1, inplace=True)
    df.drop(ind_coluna, axis=0, inplace=True)

    df.drop(ind_linha, axis=0, inplace=True)
    df.drop(ind_linha, axis=1, inplace=True)

    df[juntar] = ''
    df.loc[juntar] = ''

    df.sort_index(axis=1, inplace=True)
    df.sort_index(axis=0, inplace=True)

    #Fazendo reformulação da matrix
    for coluna in df.columns:
        for linha in df.index:

            separar_coluna = coluna.split(',')
            separar_linha = linha.split(',')
            
            #Calculando distancia de mais de 2 individuo
            if (len(separar_coluna) > 1) and (len(separar_linha) > 1):

                lista = []
                for i in separar_coluna:
                    combinacoes = list(itertools.product([i], list(separar_linha)))

                    for comb in combinacoes:
                        colunazinha = list(comb)[0]
                        linhazinha = list(comb)[1]
                        valor = df_completo[colunazinha][linhazinha]
                        lista.append(valor)

                media = statistics.mean(lista)
                df[coluna][linha] = round(media, 2)
            
            #Calculando distancia de 3 individuos
            elif len(separar_coluna) > 1:
                lista = []
                for i in separar_coluna:
                    lista.append(df_completo[i][linha])

                media = statistics.mean(lista)
                df[coluna][linha] = round(media, 2)

            elif len(separar_linha) > 1:
                lista = []
                for i in separar_linha:
                    lista.append(df_completo[coluna][i])

                media = statistics.mean(lista)
                df[coluna][linha] = round(media, 2)
            
            #Calculando distancia de 2 individuos
            else:
                df[coluna][linha] = df_completo[coluna][linha]

                
    #Tirando as diagonais e repetidos do lado superior direito
    dff = df.copy()
    
    lista = []
    for coluna in dff.columns:
        for linha in dff.index:
            if coluna == linha:
                dff[coluna][linha] = ''

            if sorted(list([linha, coluna])) in lista:
                dff[coluna][linha] = ''
            else:
                lista.append(sorted(list([linha, coluna])))            
                
                
    
    #display(df)
    dff.to_excel(writer, sheet_name=str(count))
    
cofenetica.to_excel(writer, sheet_name="tabela cofenetica")
    
#writer.save()

In [11]:
cofenetica

,Passo,Junção,Nivel
0,1,a01 - a02,0.00
1,2,"a01,a02 - a03",0.58
2,3,"a01,a02,a03 - a04",0.85
3,4,a05 - a06,0.97
4,5,"a05,a06 - a07",1.23
5,6,"a01,a02,a03,a04 - a05,a06,a07",1.60


In [12]:
dfc = pd.DataFrame(index=fonte.index, columns = list(fonte.index))

for coluna in dfc.columns:
    for linha in dfc.index:
        for i in range(len(cofenetica)-1, -1, -1):
            if (coluna in cofenetica['Junção'][i]) and (linha in cofenetica['Junção'][i]):
                dfc[coluna][linha] = cofenetica['Nivel'][i]
                
display(df_completo)                
display(dfc)

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a02,0.0,0.0,0.58,0.71,1.54,0.86,2.19
a03,0.58,0.58,0.0,1.12,2.06,1.44,2.77
a04,0.71,0.71,1.12,0.0,0.99,0.85,1.91
a05,1.54,1.54,2.06,0.99,0.0,0.97,1.1
a06,0.86,0.86,1.44,0.85,0.97,0.0,1.35
a07,2.19,2.19,2.77,1.91,1.1,1.35,0.0


,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,0.58,0.85,1.6,1.6,1.6
a02,0.0,0.0,0.58,0.85,1.6,1.6,1.6
a03,0.58,0.58,0.58,0.85,1.6,1.6,1.6
a04,0.85,0.85,0.85,0.85,1.6,1.6,1.6
a05,1.6,1.6,1.6,1.6,0.97,0.97,1.23
a06,1.6,1.6,1.6,1.6,0.97,0.97,1.23
a07,1.6,1.6,1.6,1.6,1.23,1.23,1.23


In [13]:
lista = []
for coluna in df_completo.columns:
    for linha in df_completo.index:
        if coluna == linha:
            df_completo[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            df_completo[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna])))
            
display(df_completo)

lista = []
for coluna in dfc.columns:
    for linha in dfc.index:
        if coluna == linha:
            dfc[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            dfc[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna])))
            
display(dfc)

dfc.to_excel(writer, sheet_name="matrix cofenetica")

,a01,a02,a03,a04,a05,a06,a07
a01,,,,,,,
a02,0.0,,,,,,
a03,0.58,0.58,,,,,
a04,0.71,0.71,1.12,,,,
a05,1.54,1.54,2.06,0.99,,,
a06,0.86,0.86,1.44,0.85,0.97,,
a07,2.19,2.19,2.77,1.91,1.1,1.35,


,a01,a02,a03,a04,a05,a06,a07
a01,,,,,,,
a02,0.0,,,,,,
a03,0.58,0.58,,,,,
a04,0.85,0.85,0.85,,,,
a05,1.6,1.6,1.6,1.6,,,
a06,1.6,1.6,1.6,1.6,0.97,,
a07,1.6,1.6,1.6,1.6,1.23,1.23,


In [14]:
cor = pd.DataFrame(columns = ['S','C'])

for coluna in dfc.columns:
    for linha in dfc.index:
        if dfc[coluna][linha] != '':
            cor.loc[','.join(list([coluna, linha]))] = [df_completo[coluna][linha], dfc[coluna][linha]]
            
cor

cor.to_excel(writer, sheet_name="tabela correlação"

,S,C
"a01,a02",0.00,0.00
"a01,a03",0.58,0.58
"a01,a04",0.71,0.85
"a01,a05",1.54,1.60
"a01,a06",0.86,1.60
"a01,a07",2.19,1.60
"a02,a03",0.58,0.58
"a02,a04",0.71,0.85
"a02,a05",1.54,1.60
"a02,a06",0.86,1.60


In [15]:
cor['S'].corr(cor['C'])

0.7039411307650045